In [2]:
# Here are some imports that are used along this notebook
import math
import itertools
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from time import time
import tensorflow as tf
from collections import OrderedDict
import keras
from sklearn.model_selection import KFold
from keras.utils import np_utils
%matplotlib inline
gt0 = time()

Using TensorFlow backend.


In [3]:
train_nsl_kdd_dataset_path = "NSL_KDD_Dataset/KDDTrain+.txt"
test_nsl_kdd_dataset_path = "NSL_KDD_Dataset/KDDTest+.txt"

In [4]:
col_names = np.array(["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","labels"])

In [7]:
attack_dict_five_class = {
    'normal': 'normal',
    
    'back': 'DoS',
    'land': 'DoS',
    'neptune': 'DoS',
    'pod': 'DoS',
    'smurf': 'DoS',
    'teardrop': 'DoS',
    'mailbomb': 'DoS',
    'apache2': 'DoS',
    'processtable': 'DoS',
    'udpstorm': 'DoS',
    
    'ipsweep': 'Probe',
    'nmap': 'Probe',
    'portsweep': 'Probe',
    'satan': 'Probe',
    'mscan': 'Probe',
    'saint': 'Probe',

    'ftp_write': 'R2L',
    'guess_passwd': 'R2L',
    'imap': 'R2L',
    'multihop': 'R2L',
    'phf': 'R2L',
    'spy': 'R2L',
    'warezclient': 'R2L',
    'warezmaster': 'R2L',
    'sendmail': 'R2L',
    'named': 'R2L',
    'snmpgetattack': 'R2L',
    'snmpguess': 'R2L',
    'xlock': 'R2L',
    'xsnoop': 'R2L',
    'worm': 'R2L',
    
    'buffer_overflow': 'U2R',
    'loadmodule': 'U2R',
    'perl': 'U2R',
    'rootkit': 'U2R',
    'httptunnel': 'U2R',
    'ps': 'U2R',    
    'sqlattack': 'U2R',
    'xterm': 'U2R'
}


attack_two_class = []


for key in attack_dict_five_class.keys():
    if key == 'normal':
        pass
    else:
         attack_two_class.append(key)

In [9]:
#attack_dict_five_class.keys()
#len(keys)

In [10]:
#Load test and train data
train_df = pd.read_csv(train_nsl_kdd_dataset_path, names = col_names)
test_df  = pd.read_csv(test_nsl_kdd_dataset_path , names = col_names)
train_labels = train_df.pop('dst_host_srv_rerror_rate')
test_labels = test_df.pop('dst_host_srv_rerror_rate')
total_dataset = pd.concat([train_df, test_df])
#print(total_dataset.shape)


In [11]:

total_dataset.head()
total_dataset.shape
#train_df



(148517, 41)

In [12]:
#print(pd.get_dummies(train_df, columns=['flag']))
#get = train_df.pop('protocol_type')
#print(get)
#pd.get_dummies(get)

In [13]:
#test_df.head(),train_df.head()
    

In [14]:
total_dataset = pd.get_dummies(total_dataset)
#print(total_dataset.shape)
#train_size = int(len(total_dataset) * 0.80)
#print(train_size)
#test_size = len(total_dataset) - train_size
#print(test_size)
train_df = total_dataset.iloc[0:125973, :]
test_df = total_dataset.iloc[125973:, :]
print(train_df.shape, test_df.shape)

(125973, 122) (22544, 122)


In [15]:
test_labels_for_two_class = pd.DataFrame(test_labels.as_matrix(), columns=["class"])

In [16]:
train_labels_for_two_class = pd.DataFrame(train_labels.as_matrix(), columns=["class"])
#print(train_labels_for_two_class)
test_labels_for_two_class = pd.DataFrame(test_labels.as_matrix(), columns=["class"])

In [17]:
train_labels_for_two_class.loc[train_labels_for_two_class['class'].isin(attack_two_class) , 'class'] = 1
train_labels_for_two_class.loc[train_labels_for_two_class['class'] == "normal" , 'class'] = 0
#print(train_labels_for_two_class)
#for test labels 
test_labels_for_two_class.loc[test_labels_for_two_class['class'].isin(attack_two_class) , 'class'] = 1
test_labels_for_two_class.loc[test_labels_for_two_class['class'] == "normal" , 'class'] = 0




In [18]:
train_labels_for_two_class = np_utils.to_categorical(train_labels_for_two_class)
print(train_labels_for_two_class.shape)
test_labels_for_two_class = np_utils.to_categorical(test_labels_for_two_class)
print(train_labels_for_two_class.shape)

(125973, 2)
(125973, 2)


In [19]:
#min-max mazimazation for the data
from sklearn.preprocessing import MinMaxScaler
train_X = train_df.as_matrix()
train_Y = train_labels_for_two_class

test_X = test_df.as_matrix()
test_Y = test_labels_for_two_class
scaler = MinMaxScaler() 
scaler.fit_transform(train_X)
scaler.fit_transform(train_Y)
scaler.fit_transform(test_X)
scaler.fit_transform(test_Y)
#train_X.shape[1]
test_X.shape
    

(22544, 122)

In [20]:
X_test = test_X[:1,]

In [47]:
X_test

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.29000000e+02,   1.00000000e+01,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          4.00000000e-02,   6.00000000e-02,   0.00000000e+00,
          2.55000000e+02,   1.00000000e+01,   4.00000000e-02,
          6.00000000e-02,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   2.10000000e+01,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        

In [23]:
#Training our auto encoder

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras import regularizers



In [24]:
#print(train_X)


In [25]:
input_dim = train_X.shape[1]
input_layer = Input(shape=(input_dim, ))
encoded = Dense(30, activation="relu", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
autoencoder = Model(inputs=input_layer, outputs=decoded)
encoder = Model(input_layer, encoded)
encoded_input = Input(shape=(30,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input,decoder_layer(encoded_input))

In [26]:
nb_epoch = 10
batch_size = 128
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])

autoencoder.fit(train_X,train_X ,
                    epochs=nb_epoch,
                    verbose = 1,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_X, test_X))

Train on 125973 samples, validate on 22544 samples
Epoch 1/10
125973/125973 [==============================] - 4s 31us/step - loss: 415028351736.1074 - acc: 0.2523 - val_loss: 1836718921.7161 - val_acc: 0.2283
Epoch 2/10
125973/125973 [==============================] - 4s 28us/step - loss: 415028348308.0401 - acc: 0.1935 - val_loss: 1836718910.6870 - val_acc: 0.2760
Epoch 3/10
125973/125973 [==============================] - 4s 29us/step - loss: 415028354407.8692 - acc: 0.2354 - val_loss: 1836718922.0063 - val_acc: 0.2338
Epoch 4/10
125973/125973 [==============================] - 4s 28us/step - loss: 415028355793.4564 - acc: 0.3037 - val_loss: 1836718919.8385 - val_acc: 0.2363
Epoch 5/10
125973/125973 [==============================] - 4s 28us/step - loss: 415028350121.8682 - acc: 0.3242 - val_loss: 1836718919.1011 - val_acc: 0.2463
Epoch 6/10
125973/125973 [==============================] - 4s 28us/step - loss: 415028353227.0989 - acc: 0.3336 - val_loss: 1836718918.6960 - val_acc: 0.

In [27]:
X = encoder.predict(train_X)
#x_train
X.shape

(125973, 30)

In [28]:
Y = decoder.predict(X)
Y.shape

(125973, 122)

In [29]:
input_dim_1 = X.shape[1]
input_layer_1 = Input(shape=(input_dim_1, ))
encoded_1 = Dense(20, activation="relu", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer_1)
decoded_1 = Dense(input_dim_1, activation='sigmoid')(encoded_1)
autoencoder_1 = Model(inputs=input_layer_1, outputs=decoded_1)
encoder_1 = Model(input_layer_1, encoded_1)
encoded_input_1 = Input(shape=(20,))
decoder_layer_1 = autoencoder_1.layers[-1]
decoder_1 = Model(encoded_input_1,decoder_layer_1(encoded_input_1))

In [31]:
nb_epoch = 10
batch_size = 128
autoencoder_1.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])

history = autoencoder_1.fit(X,X,
                    epochs=nb_epoch,
                    verbose = 1,
                    batch_size=batch_size,
                    shuffle=True)

Epoch 1/10
125973/125973 [==============================] - 2s 14us/step - loss: 80917005.2277 - acc: 0.8623
Epoch 2/10
125973/125973 [==============================] - 2s 12us/step - loss: 80917000.9988 - acc: 0.8970
Epoch 3/10
125973/125973 [==============================] - 2s 13us/step - loss: 80916999.8879 - acc: 0.9162
Epoch 4/10
125973/125973 [==============================] - 2s 13us/step - loss: 80916999.5914 - acc: 0.9335
Epoch 5/10
125973/125973 [==============================] - 2s 12us/step - loss: 80916999.3588 - acc: 0.9341
Epoch 6/10
125973/125973 [==============================] - 2s 12us/step - loss: 80916999.1872 - acc: 0.9315
Epoch 7/10
125973/125973 [==============================] - 2s 12us/step - loss: 80916999.5061 - acc: 0.9074
Epoch 8/10
125973/125973 [==============================] - 2s 12us/step - loss: 80916999.3123 - acc: 0.8890
Epoch 9/10
125973/125973 [==============================] - 2s 12us/step - loss: 80916999.4598 - acc: 0.8936
Epoch 10/10
125973/

In [32]:
Z = encoder_1.predict(X)
Z.shape

(125973, 20)

In [33]:
# input_layer = Input(shape=(train_X.shape[1], ))








# # encoded = Dense(30, activation="relu", 
# #                 activity_regularizer=regularizers.l1(10e-5))(input_layer)

# # encoder = Model(input_layer, encoded)
# # encoded_1 = Dense(20, activation="relu", 
# #                 activity_regularizer=regularizers.l1(10e-5))(encoder.output)
# # encoder_1 = Model(, encoded_1)




# # encoded_2 = Dense(10, activation='relu')(input_layer_2)
# # encoder_2 = Model(input_layer_2, encoded_2)
# out2 = Dense(2, activation='softmax')(encoder_2.output)
# newmodel = Model(encoder_2.input,out2)

In [34]:
# from keras import optimizers
# opt = optimizers.SGD(lr=0.00001)
# newmodel.compile(loss='categorical_crossentropy',
#           optimizer=opt, 
#           metrics=['accuracy']) 

# newmodel.fit(train_X, train_Y,
#       epochs=10,
#       batch_size=128,
#       shuffle=True,
#       validation_data=(test_X, test_Y))

In [35]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Layer
from keras import optimizers


In [36]:
model = Sequential()
model.add(encoder)
model.add(encoder_1)
model.add(Dense(2, activation='softmax'))

In [37]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [38]:
scores = model.evaluate(test_X, test_Y, verbose=1, steps=50) 
print("Accuracy: ", scores[1])

50/50 [==============================] - 2s 49ms/step
Accuracy:  0.430713266134


In [39]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.activations import sigmoid
from keras.utils import np_utils

In [40]:
model.fit(train_X, train_Y, batch_size=150, nb_epoch=100, validation_data=(test_X, test_Y))

C:\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 125973 samples, validate on 22544 samples
Epoch 1/100
125973/125973 [==============================] - 3s 21us/step - loss: 0.2807 - acc: 0.9335 - val_loss: 0.7889 - val_acc: 0.7584
Epoch 2/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1848 - acc: 0.9686 - val_loss: 0.9019 - val_acc: 0.7264
Epoch 3/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.3545 - acc: 0.9705 - val_loss: 0.8397 - val_acc: 0.7378
Epoch 4/100
125973/125973 [==============================] - 2s 20us/step - loss: 0.1630 - acc: 0.9713 - val_loss: 0.8025 - val_acc: 0.7713
Epoch 5/100
125973/125973 [==============================] - 2s 20us/step - loss: 0.1588 - acc: 0.9717 - val_loss: 0.8318 - val_acc: 0.7741
Epoch 6/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.2608 - acc: 0.9716 - val_loss: 0.8247 - val_acc: 0.7670
Epoch 7/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1524 - acc: 0.9726 - val_

Epoch 59/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1480 - acc: 0.9654 - val_loss: 0.9085 - val_acc: 0.6966
Epoch 60/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1435 - acc: 0.9667 - val_loss: 0.9533 - val_acc: 0.7154
Epoch 61/100
125973/125973 [==============================] - 2s 18us/step - loss: 0.1409 - acc: 0.9676 - val_loss: 0.9374 - val_acc: 0.6996
Epoch 62/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1484 - acc: 0.9651 - val_loss: 0.9649 - val_acc: 0.7083
Epoch 63/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1410 - acc: 0.9680 - val_loss: 0.9492 - val_acc: 0.6974
Epoch 64/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1397 - acc: 0.9678 - val_loss: 0.9892 - val_acc: 0.7048
Epoch 65/100
125973/125973 [==============================] - 2s 19us/step - loss: 0.1461 - acc: 0.9656 - val_loss: 0.9724 - val_acc: 0.6963
Epoch 66/100


In [41]:
scores = model.evaluate(test_X, test_Y, verbose=1, steps=50) 
print("Accuracy: ", scores[1])

50/50 [==============================] - 2s 43ms/step
Accuracy:  0.683596551418


In [42]:
scores = model.evaluate(train_X, train_Y, verbose=1, steps=50) 
print("Accuracy: ", scores[1])

50/50 [==============================] - 13s 250ms/step
Accuracy:  0.954045712948


In [50]:
x=model.predict_classes(test_X)
for i in range(len(x)):
    if(x[i]==1):
        print("Attack")
    else:
        print("Normal")

Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Attack
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Attack
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack

Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Normal
Normal
Normal

Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack

Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Attack
Attack
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Attack
Normal
Attack
Attack
Normal
Attack
Normal

Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal

...

Normal
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Normal
Attack
Normal
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal

Normal
Normal
Attack
Attack
Normal
Attack
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Attack
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal

Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Attack

Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Attack
Attack
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal

Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Attack
Attack
Attack
Normal
Attack
Normal
Attack
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Attack
Attack
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Attack
Normal
Attack
Normal
Normal
Normal
Attack
Normal
Normal
Attack
Normal
Normal
Normal
Normal
Normal
Attack

In [44]:
print(x)

[1]
